In [2]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [3]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [4]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [5]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [6]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [12]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [14]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.930s

OK


In [15]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [ ]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

In [ ]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [17]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.726s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [32]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address


# get the private key from the exercise in Chapter 4
# change address should be the address generated from Chapter 4

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence

# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee

# create a transaction output for the target amount and address
# create a transaction output for the change amount and address
# create the transaction object

# sign the one input in the transaction object using the private key
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx


target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
my_address = 'mkt4ofnojUzBfyZzdbmvASxaT3KwxtY29g'

prev_tx = bytes.fromhex('110e1138db767642e32c0b21ef94c968c18e56b2a9ac2bcd5981d83cc29da509')
prev_index = 0
in1 = TxIn(prev_tx, prev_index)

target_amount = int(0.6 * in1.value(True))
fee = int(0.01 * in1.value(True))
change = int(in1.value(True) - target_amount - fee)

hash_target_out1 = decode_base58(target_address)
scriptPubKey_out1 = p2pkh_script(hash_target_out1)
out1 = TxOut(target_amount, scriptPubKey_out1)

hash_change_out2 = decode_base58(my_address)
scriptPubKey_out2 = p2pkh_script(hash_change_out2)
out2 = TxOut(change, scriptPubKey_out2)

ins = [in1]
outs = [out1, out2]

tx_obj = Tx(1, ins, outs, 0, True)

secret=26920121782238591022628859483324191290857752025244922875469214731595532867627
private_key = PrivateKey(secret)
sec = private_key.point.sec()

z = tx_obj.sig_hash(0)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
script_sig = Script([sig, sec])
tx_obj.tx_ins[0].script_sig = script_sig

#print(tx_obj.verify())

print(tx_obj.serialize().hex())







010000000109a59dc23cd88159cd2baca9b2568ec168c994ef210b2ce3427676db38110e11000000006b483045022100cfa503242123d2307e1812d1485807b795e52940ffde4eac5162eb8e9407e0a302206dc6f5eb5e2591be8a8c33333dec084ea13aaeb9edca05b1b848502d269fad510121031e48607ac71580d029b2b043fc78fcc0092a56b8d965d700dbd1a7a6556a220affffffff02004b0000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788acc0300000000000001976a9143ad6cc84e7307c71e203ed0271598eba0c6bd19388ac00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [37]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above



# get the private key from the exercise in Chapter 4

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create the first transaction input with the default ScriptSig and
# sequence
# get the prev_tx and prev_index from the transaction in Exercise 4
# create the second transaction input with the default ScriptSig and
# sequence

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee

# create a transaction output for the amount and address

# sign the first input using the private key
# sign the second input using the private key 
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx

my_address = 'mkt4ofnojUzBfyZzdbmvASxaT3KwxtY29g'

prev_tx = bytes.fromhex('2235fab2e09645c4e83058b02b53a8e79d4303ecd9201ec6c2ae12a5da3f11d0')
prev_index = 1
in1 = TxIn(prev_tx, prev_index)

prev_tx = bytes.fromhex('83849d967649cd8b79f1bfd6ca913a9ea64e4685b418ade341d6d0be49f4eab1')
prev_index = 1
in2 = TxIn(prev_tx, prev_index)

input_value = in1.value(True) + in2.value(True)
target_amount = int(0.98 * input_value)
fee = int(input_value - target_amount)

hash160 = decode_base58(my_address)
scriptPubKey  = p2pkh_script(hash160)
out1 = TxOut(target_amount, scriptPubKey)

tx_obj = Tx(1, [in1,in2], [out1], 0, True)

secret=26920121782238591022628859483324191290857752025244922875469214731595532867627
private_key = PrivateKey(secret)
sec = private_key.point.sec()

tx_obj.sign_input(0, private_key)
tx_obj.sign_input(1, private_key)
print(tx_obj.verify())

print(tx_obj.serialize().hex())




True
0100000002d0113fdaa512aec2c61e20d9ec03439de7a8532bb05830e8c44596e0b2fa3522010000006b483045022100bdec10478c561410d2203715ee1748819f00a9ee6175b48cb67b5d080bdf190a02207e02a5d0e5515ec8460d312089306a1f3082cee9b85afbb19df161326b6968a00121031e48607ac71580d029b2b043fc78fcc0092a56b8d965d700dbd1a7a6556a220affffffffb1eaf449bed0d641e3ad18b485464ea69e3a91cad6bff1798bcd4976969d8483010000006a47304402204156545d3c62d1b2a7be6eb90e452966c75a94f09b9f149ef0944f251b4711d4022024ddbfc3c4c670413355198f316e2d77a9563a7f2c46ffff0af38ba1308b2e8e0121031e48607ac71580d029b2b043fc78fcc0092a56b8d965d700dbd1a7a6556a220affffffff01567c0000000000001976a9143ad6cc84e7307c71e203ed0271598eba0c6bd19388ac00000000
